In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import boto3
from botocore.client import Config
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

os.chdir('C:\\Users\\A Sua\\Documents\\AWS\\credentials')
list_of_lines = []
with open('capstoner_key.txt') as fp:
    lines = fp.readlines()
    for line in lines:
        line = line.strip()
        list_of_lines.append(line)
    fp.close()


location = list_of_lines[0].split('=')[1]
AWS_ACCESS_KEY_ID = list_of_lines[1].split('=')[1]
AWS_SECRET_ACCESS_KEY = list_of_lines[2].split('=')[1]
os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')

### import quarterly data 

In [8]:
### BUCKETS & KEYS
BUCKET_NAME = 'capstonefiu2020-data'
key_location = 'data/processed'
file_name = 'compiled_quarterly_final_v2.csv'
key_path = key_location + '/' + file_name

### ACTIVATE CLIENT
client = boto3.client('s3', 
              aws_access_key_id = AWS_ACCESS_KEY_ID,
               aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


### INSTANTIATE DATAFRAME
csv_obj = client.get_object(Bucket = BUCKET_NAME, Key=key_path)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

df_quarter = pd.read_csv(StringIO(csv_string))
df_quarter.index = df_quarter['Quarter end']
df_quarter = df_quarter.drop('Quarter end', axis=1)
df_quarter.index.names = ['qend']


#df_bidlo.head()
#### TRANSPOSE DATA FOR EASE OF USE
# df_quarter = df_quarter.T
df_quarter.index = pd.to_datetime(df_quarter.index)
# print(df_quarter.shape)
df_quarter.head()

,Shares split adjusted,Price,EPS basic,EPS diluted,Dividend per share,Dividend payout ratio,P/E ratio,Long-term debt to equity ratio,Symbol,Quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3
2019-06-30,67547497,57.23,0.63,0.62,0.035,0.0464,19.94,0.1880,AAN,2019Q2
2019-03-31,67677449,48.63,0.83,0.82,0.035,0.0459,17.49,0.2251,AAN,2019Q1
2018-12-31,67202919,47.00,0.90,0.89,0.040,0.0455,10.83,0.1937,AAN,2018Q4
2018-09-30,68594784,48.86,0.64,0.62,0.030,0.0267,12.00,0.1686,AAN,2018Q3


### Drop Duplicates

In [12]:
print('old shape {}'.format(df_quarter.shape))
df_quarter = df_quarter.drop_duplicates()
print('new shape: {}'.format(df_quarter.shape))

old shape (62252, 10)
new shape: (62252, 10)


### Change Columns

In [11]:
df_quarter.columns = ['sharesout', 'price', 'eps_basic', 'eps_diluted', 'div_share', 'div_payout', 'p/e', 'debt/eq', 'symbol', 'quarter']
df_quarter.head(1)

,sharesout,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3


### Change symbol to string

In [15]:
df_quarter['symbol'] = df_quarter['symbol'].astype(str)
df_quarter['quarter'] = df_quarter['quarter'].astype(str)

### Change numerical values to Int. Float.

In [16]:
df_quarter['sharesout'] = df_quarter['sharesout'].astype(int)
df_quarter['price'] = df_quarter['price'].astype(float)
df_quarter['eps_basic'] = df_quarter['eps_basic'].astype(float)
df_quarter['eps_diluted'] = df_quarter['eps_diluted'].astype(float)
df_quarter['div_share'] = df_quarter['div_share'].astype(float)
df_quarter['div_payout'] = df_quarter['div_payout'].astype(float)
df_quarter['p/e'] = df_quarter['p/e'].astype(float)
df_quarter['debt/eq'] = df_quarter['debt/eq'].astype(float)

In [18]:
df_quarter.head(1)

,sharesout,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3


### Link up the Quarter to the Dates

In [72]:
df_quarter['new_key'] = df_quarter['symbol'] + '_'+ df_quarter['quarter']

In [69]:
dates_table = pd.read_csv('data\\external\\std_dates.csv')
dates_table['std_date']  = pd.to_datetime(dates_table['std_date'])
dates_table.head()

,Quarter,std_date,index
0,2018Q4,2018-12-31,AAN_2018Q4
1,2018Q3,2018-09-30,AAN_2018Q3
2,2018Q2,2018-06-30,AAN_2018Q2
3,2018Q1,2018-03-31,AAN_2018Q1
4,2017Q4,2017-12-31,AAN_2017Q4


In [75]:
df_merge = pd.merge(dates_table, df_quarter, left_on='index', right_on='new_key', how='inner')

### Filter Out Symbols That only traded from 1/1/2007 - 12/31/2018

In [25]:
df_07_18 = df_quarter[df_quarter['quarter'].str.contains('2007') | df_quarter['quarter'].str.contains('2008')| df_quarter['quarter'].str.contains('2009')|
           df_quarter['quarter'].str.contains('2010') | df_quarter['quarter'].str.contains('2011')| df_quarter['quarter'].str.contains('2012')|
           df_quarter['quarter'].str.contains('2013') | df_quarter['quarter'].str.contains('2014')| df_quarter['quarter'].str.contains('2015')|
           df_quarter['quarter'].str.contains('2016') | df_quarter['quarter'].str.contains('2017')| df_quarter['quarter'].str.contains('2018')]

In [76]:
# df_07_18 = df_quarter[(df_quarter.index >= '2007-1-1') & (df_quarter.index <= '2018-12-31')]
# print(df_07_18.shape)

In [36]:
#df_07_18.index.sort_values()

In [77]:
symbol_list = df_merge['symbol'].unique()

### Filter out the companies that had earnings data from 2007 - 2018 (PRICE)

In [91]:
symbol_error = []
df_reduced = df_merge[['std_date', 'symbol', 'price']]
########----------------#######
df_newq = pd.DataFrame([])

symbol = 'AAN'
for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['price']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,18.83,18.81,16.62,13.69,12.12,15.61,17.17,14.67,16.29,...,28.30,29.50,34.69,42.68,39.67,42.98,43.87,48.86,47.00,NaN
AAPL,NaN,12.84,15.52,19.05,25.25,22.56,23.91,21.54,14.21,13.44,...,111.38,129.63,148.36,153.68,164.83,166.87,177.41,206.54,190.03,NaN
AAP,NaN,37.85,41.42,34.83,36.13,34.59,37.66,40.68,30.77,37.07,...,155.96,158.82,125.42,98.81,93.30,111.81,122.78,155.00,167.26,NaN
AA,NaN,96.21,114.79,118.53,110.88,99.54,117.63,84.89,43.72,26.12,...,24.43,24.66,25.20,24.76,25.30,26.87,20.27,19.88,19.62,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60.05,63.03,68.39,80.16,92.17,108.49,97.73,93.50,87.05,NaN


In [118]:
df_newq = df_newq.iloc[:, 1:-1]

In [147]:
new_symbols_price = []
null_symbols_price = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_price.append(symbol)
    else:
        null_symbols_price.append(symbol)

In [148]:
len(new_symbols_price), len(null_symbols_price)

(515, 246)

In [146]:
df_quarter_price = pd.DataFrame([])
for symbol in new_symbols:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_price = pd.concat([df_quarter_price, temp_df])

df_quarter_price

### EPS (Basic)

In [150]:
#symbol_error = []
df_reduced = df_merge[['std_date', 'symbol', 'eps_basic']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['eps_basic']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,0.54,0.36,0.29,0.29,0.46,0.44,0.40,0.39,0.66,...,0.31,0.75,0.51,0.36,2.51,0.75,0.55,0.64,0.90,NaN
AAPL,NaN,0.89,0.94,1.04,1.81,1.19,1.21,1.27,1.81,1.35,...,3.38,2.11,1.68,2.08,3.92,2.75,2.36,2.94,4.22,NaN
AAP,NaN,0.72,0.64,0.58,0.35,0.86,0.79,0.59,0.26,0.99,...,0.83,1.46,1.18,1.30,2.50,1.85,1.59,1.57,0.73,NaN
AA,NaN,0.76,0.82,0.64,0.76,0.37,0.67,0.33,-1.48,-0.61,...,-2.92,0.69,0.44,0.23,-1.51,0.30,0.25,0.33,0.45,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.86,1.07,1.20,1.02,0.02,1.74,1.26,1.81,-1.22,NaN


In [151]:
df_newq = df_newq.iloc[:, 1:-1]

In [152]:
new_symbols_eps = []
null_symbols_eps = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_eps.append(symbol)
    else:
        null_symbols_eps.append(symbol)

In [153]:
len(new_symbols_eps), len(null_symbols_eps)

(528, 233)

In [155]:
df_quarter_eps = pd.DataFrame([])
for symbol in new_symbols_eps:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_eps = pd.concat([df_quarter_eps, temp_df])

df_quarter_eps.shape

(528, 48)

### Div per Share

In [156]:
#symbol_error = []
df_reduced = df_merge[['std_date', 'symbol', 'div_share']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['div_share']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,0.01,0.01,0.01,0.01,0.01067,0.01067,0.01067,0.01,0.01133,...,0.03,0.0275,0.0275,0.0275,0.03,0.03,0.03,0.03,0.04,NaN
AAPL,NaN,0.00,0.00,0.00,0.00,0.00000,0.00000,0.00000,0.00,0.00000,...,0.57,0.5700,0.6300,0.6300,0.63,0.63,0.73,0.73,0.73,NaN
AAP,NaN,0.06,0.06,0.06,0.06,0.06000,0.06000,0.06000,0.06,0.06000,...,0.06,0.0600,0.0600,0.0600,0.06,0.06,0.06,0.06,0.06,NaN
AA,NaN,0.17,0.17,0.17,0.17,0.17000,0.17000,0.17000,0.17,0.17000,...,7.76,0.0600,0.0600,0.0600,0.06,0.06,0.06,0.06,0.06,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.57,0.6400,0.6400,0.6400,0.64,0.71,0.96,0.96,1.32,NaN


In [157]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape

(761, 48)

In [158]:
new_symbols_div = []
null_symbols_div = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_div.append(symbol)
    else:
        null_symbols_div.append(symbol)

In [159]:
len(new_symbols_div), len(null_symbols_div)

(532, 229)

In [160]:
df_quarter_div = pd.DataFrame([])
for symbol in new_symbols_div:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_div = pd.concat([df_quarter_div, temp_df])

df_quarter_div.shape

(532, 48)

In [169]:
df_quarter_div.shape

(532, 48)

### Price/Earning

In [170]:
#symbol_error = []
df_reduced = df_merge[['std_date', 'symbol', 'p/e']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['p/e']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,19.35,17.97,16.19,13.60,12.63,17.09,17.89,14.48,14.90,...,14.89,15.45,17.61,21.89,20.88,10.61,10.86,12.00,10.83,NaN
AAPL,NaN,32.57,34.38,37.67,44.97,34.71,34.58,29.51,18.56,17.49,...,13.45,15.51,17.33,17.44,17.94,17.17,17.14,18.74,16.01,NaN
AAP,NaN,17.52,18.91,15.55,16.06,15.24,15.56,15.83,11.88,14.89,...,25.61,25.62,22.72,19.72,19.52,17.42,18.06,21.47,22.36,NaN
AA,NaN,12.48,14.55,15.25,14.16,11.13,15.08,11.55,6.78,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.19,17.36,17.72,19.70,22.43,32.88,24.56,23.09,17.95,NaN


In [171]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape

(761, 48)

In [172]:
new_symbols_pe = []
null_symbols_pe = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_pe.append(symbol)
    else:
        null_symbols_pe.append(symbol)

In [175]:
len(new_symbols_pe), len(null_symbols_pe)

(251, 510)

In [173]:
df_quarter_pe = pd.DataFrame([])
for symbol in new_symbols_pe:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_pe = pd.concat([df_quarter_pe, temp_df])

df_quarter_pe.shape

(251, 48)

In [174]:
df_quarter_pe.shape

(251, 48)

### Debt/Equity

In [177]:
#symbol_error = []
df_reduced = df_merge[['std_date', 'symbol', 'debt/eq']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['debt/eq']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,0.1864,0.1882,0.2274,0.2760,0.2633,0.2996,0.2074,0.1508,0.0985,...,0.2371,0.3231,0.2601,0.1741,0.1572,0.2041,0.1564,0.1686,0.1937,NaN
AAPL,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.5556,0.6304,0.6786,0.7252,0.7412,0.7989,0.8450,0.8748,0.7888,NaN
AAP,NaN,0.3615,0.2890,0.4072,0.4933,0.5826,0.4355,0.4449,0.4233,0.2365,...,0.3576,0.3550,0.3336,0.3231,0.3058,0.2946,0.2857,0.2857,0.2945,NaN
AA,NaN,0.4107,0.3733,0.3993,0.3992,0.4086,0.4074,0.5576,0.7285,0.7355,...,1.5897,1.4826,1.1954,1.1543,1.4019,1.2102,1.2259,1.1902,1.0685,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.8602,7.3081,5.6277,5.0806,6.0728,8.6986,NaN,NaN,NaN,NaN


In [178]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape

(761, 48)

In [179]:
new_symbols_de = []
null_symbols_de = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_de.append(symbol)
    else:
        null_symbols_de.append(symbol)

In [180]:
len(new_symbols_de), len(null_symbols_de)

(476, 285)

In [182]:
df_quarter_de = pd.DataFrame([])
for symbol in new_symbols_de:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_de = pd.concat([df_quarter_de, temp_df])

df_quarter_de.shape

(476, 48)

In [183]:
df_quarter_de.shape

(476, 48)

In [184]:
df_quarter_de.head()

std_date,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,2009-06-30,...,2016-09-30,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31
AAN,0.1864,0.1882,0.2274,0.2760,0.2633,0.2996,0.2074,0.1508,0.0985,0.0970,...,0.3405,0.2371,0.3231,0.2601,0.1741,0.1572,0.2041,0.1564,0.1686,0.1937
AAPL,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.5881,0.5556,0.6304,0.6786,0.7252,0.7412,0.7989,0.8450,0.8748,0.7888
AAP,0.3615,0.2890,0.4072,0.4933,0.5826,0.4355,0.4449,0.4233,0.2365,0.2208,...,0.3651,0.3576,0.3550,0.3336,0.3231,0.3058,0.2946,0.2857,0.2857,0.2945
AA,0.4107,0.3733,0.3993,0.3992,0.4086,0.4074,0.5576,0.7285,0.7355,0.7355,...,0.8107,1.5897,1.4826,1.1954,1.1543,1.4019,1.2102,1.2259,1.1902,1.0685
ABC,0.2929,0.3247,0.3959,0.4333,0.4151,0.4520,0.4382,0.4365,0.4203,0.4275,...,1.8175,1.8292,1.5181,1.4810,1.8317,1.6062,1.4213,1.4183,1.5380,1.4177


### UPLOAD TO AWS

In [187]:
import boto3
from botocore.client import Config
import os
#os.chdir('C:\\Users\\A Sua\\Documents\\AWS\\credentials')
list_of_lines = []
with open('C:\\Users\\A Sua\\Documents\\AWS\\credentials\\capstoner_key.txt') as fp:
    lines = fp.readlines()
    for line in lines:
        line = line.strip()
        list_of_lines.append(line)
    fp.close()

location = list_of_lines[0].split('=')[1]
AWS_ACCESS_KEY_ID = list_of_lines[1].split('=')[1]
AWS_SECRET_ACCESS_KEY = list_of_lines[2].split('=')[1]
BUCKET_NAME = 'capstonefiu2020-data'

In [190]:
df_list = [df_quarter_de, df_quarter_div, df_quarter_eps, df_quarter_pe, df_quarter_price]
df_name_list = ['df_quarter_de', 'df_quarter_div', 'df_quarter_eps', 'df_quarter_pe', 'df_quarter_price']

In [191]:
for i in range(len(df_list)):
    path_to_df = 'data\\processed\\{}_07_18.csv'.format(df_name_list[i])
    df_list[i].to_csv(path_to_df)

    ## open file
    temp_data = open(path_to_df, 'rb')

    ### instantiate s3 client
    s3 = boto3.resource('s3', 
              aws_access_key_id = AWS_ACCESS_KEY_ID,
               aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
    s3.Bucket(BUCKET_NAME).put_object(Key='data/processed/{}07_18.csv'.format(df_name_list[i]), Body=temp_data)

    ### close the file
    temp_data.close()
    os.remove(path_to_df)
    print('{} added to cloud'.format(df_name_list[i]))
print('finished uploading dataframes')

df_quarter_de added to cloud
df_quarter_div added to cloud
df_quarter_eps added to cloud
df_quarter_pe added to cloud
df_quarter_price added to cloud
finished uploading dataframes


In [168]:
len(set(df_quarter_eps.index).intersection(set(df_quarter_price.index)))

514

In [141]:
#df_new.isnull().sum().sum()

In [100]:
#df_newq.index.unique().value_counts().sort_values(ascending=False)

In [125]:
#df_07_18['symbol'].value_counts().to_csv('data\\external\\quarter_symbols.csv')